In [12]:
from collections import deque
import heapq
import math

# ==========================================
# CLASSE DO JOGO DOS OITO
# ==========================================

class PuzzleState:
    def __init__(self, state, parent=None, move=None, cost=0):
        self.state = state
        self.parent = parent
        self.move = move
        self.cost = cost

    def is_goal(self):
        return self.state == (1, 2, 3, 4, 5, 6, 7, 8, 0)

    def get_successors(self):
        successors = []

        zero_index = self.state.index(0)
        row = zero_index // 3
        col = zero_index % 3

        moves = {
            "UP": (-1, 0),
            "DOWN": (1, 0),
            "LEFT": (0, -1),
            "RIGHT": (0, 1)
        }

        for move_name, (dr, dc) in moves.items():
            new_row = row + dr
            new_col = col + dc

            if 0 <= new_row < 3 and 0 <= new_col < 3:
                new_index = new_row * 3 + new_col

                new_state = list(self.state)
                new_state[zero_index], new_state[new_index] = \
                    new_state[new_index], new_state[zero_index]

                successors.append(
                    PuzzleState(tuple(new_state), self, move_name, self.cost + 1)
                )

        return successors

    def __lt__(self, other):
        return False


# ==========================================
# FUNÇÃO PARA IMPRIMIR MATRIZ 3x3
# ==========================================

def print_board(state):
    print("-------------")
    for i in range(0, 9, 3):
        row = state[i:i+3]
        print("|", " | ".join(str(x) if x != 0 else " " for x in row), "|")
    print("-------------")


# ==========================================
# RECONSTRUIR CAMINHO
# ==========================================

def reconstruct_path(state):
    path = []
    while state.parent:
        path.append(state)
        state = state.parent
    path.reverse()
    return path


# ==========================================
# MOSTRAR PASSO A PASSO
# ==========================================

def show_solution(solution_state):
    path = reconstruct_path(solution_state)

    print("\n🔵 ESTADO INICIAL:")
    root = solution_state
    while root.parent:
        root = root.parent
    print_board(root.state)

    step = 1
    for state in path:
        print(f"\n➡ PASSO {step} — Movimento: {state.move}")
        print_board(state.state)
        step += 1

# ==========================================
# BFS
# ==========================================

def bfs(initial_state):
    queue = deque([initial_state])
    visited = set()
    nodes_visited = 0

    while queue:
        current = queue.popleft()
        nodes_visited += 1

        if current.is_goal():
            return current, nodes_visited

        visited.add(current.state)

        for successor in current.get_successors():
            if successor.state not in visited:
                queue.append(successor)

    return None, nodes_visited


# ==========================================
# HEURÍSTICA MANHATTAN
# ==========================================

def manhattan_distance(state):
    goal_positions = {
        1: (0, 0), 2: (0, 1), 3: (0, 2),
        4: (1, 0), 5: (1, 1), 6: (1, 2),
        7: (2, 0), 8: (2, 1)
    }

    distance = 0

    for i, value in enumerate(state):
        if value != 0:
            current_row = i // 3
            current_col = i % 3
            goal_row, goal_col = goal_positions[value]

            distance += abs(current_row - goal_row) + abs(current_col - goal_col)

    return distance


# ==========================================
# A*
# ==========================================

def a_star(initial_state):
    heap = []
    heapq.heappush(heap, (0, initial_state))
    visited = set()
    nodes_visited = 0

    while heap:
        _, current = heapq.heappop(heap)
        nodes_visited += 1

        if current.is_goal():
            return current, nodes_visited

        visited.add(current.state)

        for successor in current.get_successors():
            if successor.state not in visited:
                f = successor.cost + manhattan_distance(successor.state)
                heapq.heappush(heap, (f, successor))

    return None, nodes_visited


# ==========================================
# EXECUÇÃO
# ==========================================

if __name__ == "__main__":

    initial = PuzzleState((0, 8, 7,
                           6, 5, 4,
                           3, 2, 1))

    print("===== BFS =====")
    solution_bfs, visited_bfs = bfs(initial)
    print(f"Nós visitados: {visited_bfs}")
    show_solution(solution_bfs)

    print("===== A* =====")
    solution_astar, visited_astar = a_star(initial)
    print(f"Nós visitados: {visited_astar}")
    show_solution(solution_astar)

===== BFS =====
Nós visitados: 458857

🔵 ESTADO INICIAL:
-------------
|   | 8 | 7 |
| 6 | 5 | 4 |
| 3 | 2 | 1 |
-------------

➡ PASSO 1 — Movimento: DOWN
-------------
| 6 | 8 | 7 |
|   | 5 | 4 |
| 3 | 2 | 1 |
-------------

➡ PASSO 2 — Movimento: DOWN
-------------
| 6 | 8 | 7 |
| 3 | 5 | 4 |
|   | 2 | 1 |
-------------

➡ PASSO 3 — Movimento: RIGHT
-------------
| 6 | 8 | 7 |
| 3 | 5 | 4 |
| 2 |   | 1 |
-------------

➡ PASSO 4 — Movimento: RIGHT
-------------
| 6 | 8 | 7 |
| 3 | 5 | 4 |
| 2 | 1 |   |
-------------

➡ PASSO 5 — Movimento: UP
-------------
| 6 | 8 | 7 |
| 3 | 5 |   |
| 2 | 1 | 4 |
-------------

➡ PASSO 6 — Movimento: UP
-------------
| 6 | 8 |   |
| 3 | 5 | 7 |
| 2 | 1 | 4 |
-------------

➡ PASSO 7 — Movimento: LEFT
-------------
| 6 |   | 8 |
| 3 | 5 | 7 |
| 2 | 1 | 4 |
-------------

➡ PASSO 8 — Movimento: LEFT
-------------
|   | 6 | 8 |
| 3 | 5 | 7 |
| 2 | 1 | 4 |
-------------

➡ PASSO 9 — Movimento: DOWN
-------------
| 3 | 6 | 8 |
|   | 5 | 7 |
| 2 | 1 | 4 